In [9]:
# Some useful imports
import numpy as np
import time

In [10]:
def priceAmericanPut(S0, K, T, r, N, u, d):
    dp = np.zeros((N+1, N+1))
    dt = T / N
    R = np.exp(r * dt)
    p = (R - d) / (u - d)

    for j in range(N+1):
        dp[N,j] = max(0, K - S0 * (u**j) * (d**(N-j)))

    for i in range(N-1, -1, -1):
        for j in range(i+1):
            exercise = max(K - S0 * (u**j) * (d**(i-j)), 0)
            hold = (1/R) * (p * dp[i+1][j+1] + (1-p) * dp[i+1][j])
            dp[i,j] = max(exercise, hold)

    return dp[0,0]

def priceAmericanPutVectorized(S0, K, T, r, N, u, d):
    dp = np.zeros((N+1, N+1))
    dt = T / N
    R = np.exp(r * dt)
    p = (R - d) / (u - d)

    dp[N,:] = np.maximum(0, K-S0 * u**(np.arange(0,N+1,1)) * d**(np.arange(N,-1,-1)))

    for i in range(N-1, -1, -1):
        S = S0 * u**(np.arange(0,i+1,1)) * d**(np.arange(i,-1,-1))
        exercise = np.maximum(K-S, 0)
        hold = (1/R) * (p*dp[i+1,1:i+2] + (1-p)*dp[i+1,0:i+1])
        dp[i,:i+1] = np.maximum(exercise, hold)

    return dp[0][0]

def priceAmericanPutFast(S0, K, T, r, N, u, d):
    S = S0 * u**np.arange(0,N+1,1) * d**np.arange(N,-1,-1)
    dp = np.maximum(0, K-S) # in the ith loop, dp contains Cij for j in [0,i]
    dt = T / N
    discount = np.exp(-r*dt)
    p = (np.exp(r*dt)-d) / (u-d)
    for i in range(N-1,-1,-1):
        S = S0 * u**np.arange(0,i+1,1) * d**np.arange(i,-1,-1)
        exercise = np.maximum(0, K-S)
        hold = discount * (p*dp[1:i+2] + (1-p)*dp[:i+1])
        dp = np.maximum(exercise, hold)
    return dp[0]

In [34]:
# Parameters
S0 = 100
K = 100
T = 1
r = 0.06
N = 2000
u = 1.1
d = 1 / u

# priceAmericanPut
start = time.time()
result1 = priceAmericanPut(S0, K, T, r, N, u, d)
end = time.time()
print("priceAmericanPut:", result1, " - Temps:", end - start, "sec")

# priceAmericanPutVectorized
start = time.time()
result2 = priceAmericanPutVectorized(S0, K, T, r, N, u, d)
end = time.time()
print("priceAmericanPutVectorized:", result2, " - Temps:", end - start, "sec")

# priceAmericanPutFast
start = time.time()
result3 = priceAmericanPutFast(S0, K, T, r, N, u, d)
end = time.time()
print("priceAmericanPutFast:", result3, " - Temps:", end - start, "sec")

priceAmericanPut: 93.59973577141305  - Temps: 2.4333114624023438 sec
priceAmericanPutVectorized: 93.59973577141305  - Temps: 0.09979104995727539 sec
priceAmericanPutFast: 93.59973577141305  - Temps: 0.09221506118774414 sec
